In [1]:
import pandas as pd

# 加载播放记录（用户-歌曲-播放次数）
triplets = pd.read_csv('train_triplets.txt', sep='\t', header=None, names=['user_id', 'song_id', 'plays'])
print(triplets.head())

                                    user_id             song_id  plays
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995      1
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAPDEY12A81C210A9      1
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B      2
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFNSP12AF72A0E22      1
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFOVM12A58A7D494      1


In [3]:

import pandas as pd
import h5py
def load_metadata(filename):
    with h5py.File(filename, "r") as f:
        songs = f['metadata']['songs']
        # 原有字段
        song_ids = [s.decode('utf-8', errors='ignore').strip()
                    for s in songs['song_id'][()]]
        titles   = [t.decode('utf-8', errors='ignore').strip()
                    for t in songs['title'][()]]
        # 新增作者字段
        artists  = [a.decode('utf-8', errors='ignore').strip()
                    for a in songs['artist_name'][()]]
        df = pd.DataFrame({
            'song_id':     song_ids,
            'title':       titles,
            'artist_name': artists
        })
        # 过滤掉空 song_id
        return df[df['song_id'].str.len() > 0]

metadata    = load_metadata('msd_summary_file.h5')
merged_data = pd.merge(triplets, metadata, on='song_id', how='left')

In [6]:
ShowPlaylists_num = 10


df3 = merged_data[merged_data['plays'] >= 3]

# 按用户播放总和排序，取前 10 位最“活跃”用户
top_users = df3.groupby('user_id')['plays'].sum().sort_values(ascending=False).head(ShowPlaylists_num).index.tolist()

for uid in top_users:
    print(f"\n用户 {uid} 听过的（播放>=3次）歌曲（按播放总数排序）：")
    df_u = df3[df3['user_id']==uid].sort_values(by='plays', ascending=False).head(10)
    for title, artist, cnt in zip(df_u['title'], df_u['artist_name'], df_u['plays']):
        print(f"  • {title} — {artist} — 播放 {cnt} 次")


用户 093cb74eb3c517c5179ae24caf0ebec51b24d2a2 听过的（播放>=3次）歌曲（按播放总数排序）：
  • Corrido de Boxeo — Ry Cooder — 播放 9667 次
  • Crash — Matt Willis — 播放 494 次
  • Lola Leave Your Light On — Gov't Mule — 播放 243 次
  • Que Triste Es Decir Adios — Yndio — 播放 158 次
  • Find You Waiting — DecembeRadio — 播放 152 次
  • Bubbly — Colbie Caillat — 播放 128 次
  • XXX's And OOO's (An American Girl) — Trisha Yearwood — 播放 121 次
  • Maybe — Sick Puppies — 播放 110 次
  • DONTTRUSTME [BENNYBLANCOREMIX] FEATURINGKIDCUDI (Explicit Bonus Version) — 3OH!3 — 播放 87 次
  • Andina — Strunz & Farah — 播放 69 次
  • Milo Gia Ta Pedia Mou (2006 Digital Remaster) — Viki Mosholiou — 播放 65 次
  • Bleed It Out (Album Version) — Linkin Park — 播放 65 次
  • Simarik — Tarkan — 播放 62 次
  • Potential Breakup Song — Aly & AJ — 播放 61 次
  • Prelude 12/21 — AFI — 播放 51 次
  • Pocket Porn (1990) — Renegade Soundwave — 播放 51 次
  • Kiss Me — New Found Glory — 播放 50 次
  • Bratitude — Bratz — 播放 46 次
  • Draw Me Close — Michael W. Smith — 播放 45 次
  • La